In [1]:
import sys
import os

# Einen Ordner hochgehen (du bist in /notebooks, Modell liegt eine Ebene darüber)
os.chdir("..")

# src/ zum Python-Pfad hinzufügen
sys.path.append(os.getcwd())

In [ ]:
from src.data.load_data import load_data
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import joblib

from lightgbm import LGBMRegressor

# =============================
# Konstanten
# =============================
TARGET_END = "Auftragsende_IST"
START_COL = "Auftragseingang"

DATE_COLS = [
    "Auftragseingang",
    "Auftragsende_SOLL",
    "AFO_Start_SOLL",
    "AFO_Ende_SOLL",
    "AFO_Start_IST",
    "AFO_Ende_IST",
]

# =============================
# Daten laden
# =============================
data = load_data()

print("Original Spalten:", list(data.columns))

# -----------------------------
# Datums-Spalten parsen
# -----------------------------
for col in DATE_COLS:
    data[col] = pd.to_datetime(data[col], errors="coerce")

# Target
data[TARGET_END] = pd.to_datetime(data[TARGET_END], errors="coerce")

# Valid rows
mask_valid = (~data[TARGET_END].isna()) & (~data[START_COL].isna())
data = data[mask_valid].copy()

start_dt = data[START_COL]

# =============================
# Dauer in Tagen berechnen
# =============================
duration_days = (data[TARGET_END] - start_dt).dt.total_seconds() / 86400.0
duration_days = duration_days.astype("float32")
y = duration_days

# =============================
# Date-features extrahieren
# =============================
for col in DATE_COLS:
    data[f"{col}_dow"] = data[col].dt.dayofweek
    data[f"{col}_hour"] = data[col].dt.hour
    data[f"{col}_day"] = data[col].dt.day
    data[f"{col}_month"] = data[col].dt.month
    data[f"{col}_week"] = data[col].dt.isocalendar().week.astype(int)

# Original datetime-Spalten entfernen
data = data.drop(columns=DATE_COLS + [TARGET_END])

# =============================
# ID-Spalten entfernen (vermeidet Overfitting)
# =============================
DROP_IDS = ["AuftragsID", "BauteilID", "MaschinenID"]
for col in DROP_IDS:
    if col in data.columns:
        data = data.drop(columns=[col])

# =============================
# Feature-Typen bestimmen
# =============================
categorical = data.select_dtypes(include=["object"]).columns.tolist()
numeric = data.select_dtypes(include=[np.number]).columns.tolist()

print("KATEGORIEN:", categorical)
print("NUMERISCH:", numeric)

# =============================
# Train/Test Split
# =============================
X_train, X_test, y_train, y_test, start_train_dt, start_test_dt = train_test_split(
    data, y, start_dt, test_size=0.2, random_state=42
)

# =============================
# Preprocessing
# =============================
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_pipeline, categorical),
        ("num", num_pipeline, numeric),
    ]
)

# =============================
# LightGBM Modell
# =============================
lgbm_model = LGBMRegressor(
    n_estimators=470,
    learning_rate=0.025,

    num_leaves=256,
    max_depth=10,

    min_data_in_leaf=30,

    subsample=0.85,
    colsample_bytree=0.85,

    reg_alpha=0.1,
    reg_lambda=0.3,

    n_jobs=-1,
    random_state=42
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("model", lgbm_model),
])

# =============================
# Train
# =============================
pipe.fit(X_train, y_train)

preds_days = pipe.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, preds_days)
mse = mean_squared_error(y_test, preds_days)
r2 = r2_score(y_test, preds_days)

print("\n===== MODEL PERFORMANCE =====")
print("MAE Tage:", mae)
print("MSE Tage^2:", mse)
print("R²:", r2)

# =============================
# Speichern
# =============================
out_dir = "models/lightgbm/pipeline"
os.makedirs(out_dir, exist_ok=True)

model_path = os.path.join(out_dir, "lightgbm_pipeline.pkl")
joblib.dump(pipe, model_path)

print("📦 Saved:", model_path)


Spalten im DataFrame:
['AuftragsID', 'BauteilID', 'Bauteilbezeichnung', 'Auftragseingang', 'Priorität', 'Auftragsende_SOLL', 'Arbeitsschritt', 'Arbeitsschrittbezeichnung', 'AFO_Start_SOLL', 'AFO_Ende_SOLL', 'AFO_Start_IST', 'AFO_Ende_IST', 'MaschinenID', 'Maschinenbezeichnung', 'Auftragsende_IST', 'process_type', 'possible_next_afo', 'remaining_afo_count', 'expected_total_remaining_hours', 'is_standard_sequence', 'progress_ratio']
Original Spalten: ['AuftragsID', 'BauteilID', 'Bauteilbezeichnung', 'Auftragseingang', 'Priorität', 'Auftragsende_SOLL', 'Arbeitsschritt', 'Arbeitsschrittbezeichnung', 'AFO_Start_SOLL', 'AFO_Ende_SOLL', 'AFO_Start_IST', 'AFO_Ende_IST', 'MaschinenID', 'Maschinenbezeichnung', 'Auftragsende_IST', 'process_type', 'possible_next_afo', 'remaining_afo_count', 'expected_total_remaining_hours', 'is_standard_sequence', 'progress_ratio']
KATEGORIEN: ['Bauteilbezeichnung', 'Arbeitsschrittbezeichnung', 'Maschinenbezeichnung', 'possible_next_afo']
NUMERISCH: ['Priorität', 

/Users/burak1/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30

===== MODEL PERFORMANCE =====
MAE Tage: 28.093231956267584
MSE Tage^2: 2784.5081983212776
R²: 0.8824742480996244
📦 Saved: models/lightgbm/pipeline/lightgbm_pipeline.pkl
